In [4]:
##do_runcode
##%overwritefile
##%file:src/do_jv_runcode.py
##%noruncode
    def do_runcode(self,return_code,fil_ename,class_filename,outpath,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=return_code
        fil_ename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        ##代码运行前
        mainclass=magics['package']+"."+class_filename
        self.chkjoptions(magics,'/usr/share/java/groovy.jar',outpath)
        self._write_to_stdout("The process :"+class_filename+"\n")
        ################# repl mode run code files
        #FIXME:
        if magics['runmode']=='repl':
            self._start_replprg('java',magics['joptions']+[mainclass] + magics['args'],magics)

            return_code=p.returncode
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
            return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        ############################################

    ############################################
        #################dynamically load and execute code
        #FIXME:
        # if len(magics['dlrun'])>0:
        #     p = self.create_jupyter_subprocess([self.master_path, class_filename] + magics['args'],env=self.addkey2dict(magics,'env'))
        # #################
        # else:
        cmdstr = ['java']+magics['joptions']+[mainclass] + magics['args']
        self._log(' '.join((' '+ str(s) for s in cmdstr))+"\n") 
        p = self.create_jupyter_subprocess(['java']+magics['joptions']+[mainclass] + magics['args'],env=self.addkey2dict(magics,'env'))
        self.subprocess=p
        self.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
        # if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        if len(self.addkey2dict(magics,'showpid'))>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        return_code=p.wait_end(magics)
        ##代码运行结束
        # now remove the files we have just created
        # if(os.path.exists(source_file.name)):
            # os.remove(source_file.name)
        # if(os.path.exists(class_filename)):
            # os.remove(class_filename)

        # if p.returncode != 0:
            # self._write_to_stderr("[C kernel] Executable exited with code {}".format(p.returncode))
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Java/src/do_jv_runcode.py created successfully


In [5]:
##do_compile_code
##%overwritefile
##%file:src/do_jv_compilecode.py
##%noruncode
    def do_compile_code(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=fil_ename
        sourcefilename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        
        returncode,class_filename=self._exec_javac_(fil_ename,magics)
        fil_ename=class_filename
        outpath=os.path.dirname(class_filename)
        sf = os.path.basename(class_filename)
        class_filename = sf.split(".")[0]
        return_code=returncode
        mainclass=class_filename
        
        if returncode!=0:return True,self.get_retinfo(),magics, code,fil_ename,class_filename,outpath,retstr
        # Generate executable file :end
        return bcancel_exec,retinfo,magics, code,fil_ename,class_filename,outpath,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Java/src/do_jv_compilecode.py created successfully


In [6]:
##do_create_codefile
##%overwritefile
##%file:src/do_jv_create_codefile.py
##%noruncode
    def do_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        fil_ename=''
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        class_filename=''
        outpath=''
        
        # magics['pubclass']
        # self._log(magics['pubclass']+"\n")
        source_file=self.create_codetemp_file(magics,code,suffix='.java')
        newsrcfilename=source_file.name
        fil_ename=newsrcfilename
        srcpath=os.path.dirname(source_file.name)
        sf = os.path.basename(source_file.name)

        newsrcfilename=os.path.join(srcpath,magics['pubclass']+".java")
        # self._log(newsrcfilename+"\n")
        os.rename(source_file.name ,newsrcfilename)
        fil_ename=newsrcfilename

        outpath=os.path.dirname(fil_ename)
        sf = os.path.basename(fil_ename)
        class_filename = sf.split(".")[0]
        
        return_code=True
        ############# only run gcc，no not run executable file
        if len(self.addkey2dict(magics,'onlyrungcc'))>0:
            self._log("only run gcc \n")
        return  bcancel_exec,self.get_retinfo(),magics, code,fil_ename,class_filename,outpath,retstr
            
            

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Java/src/do_jv_create_codefile.py created successfully


In [7]:
##do_preexecute
##%overwritefile
##%file:src/do_jv_preexecute.py
##%noruncode
    def do_preexecute(self,code,magics, silent, store_history=True,
                user_expressions=None, allow_stdin=False):
        bcancel_exec=False
        retinfo=self.get_retinfo()

        ##扫描代码
        #############send replcmd's command
        if magics['runmode']=='repl':
            if hasattr(self, 'replcmdwrapper'):
                if self.replcmdwrapper :
                    bcancel_exec=True
                    retinfo= self.repl_sendcmd(code, silent, store_history,
                        user_expressions, allow_stdin,magics)
                    return bcancel_exec,retinfo,magics, code

        return bcancel_exec,retinfo,magics, code

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Java/src/do_jv_preexecute.py created successfully


In [8]:
## new kernel.py
##%overwritefile
##%file:../../../jupyter-MyJava-kernel/jupyter_MyJava_kernel/kernel.py
###%file:c_kernel.py
##%noruncode
#
#   MyJava Jupyter Kernel 
#   generated by MyPython
#
##%include:../../src/head.py
##%include:../../src/common.py
##%include:../../src/Mykernel.py
class JavaKernel(MyKernel):
    implementation = 'jupyter-MyC-kernel'
    implementation_version = '1.0'
    language = 'Java'
    language_version = '18'
    language_info = {'name': 'text/java',
                     'mimetype': 'text/java',
                     'file_extension': '.java'}
    runfiletype='class'
    banner = "MyJava kernel.\n" \
             "Uses JavaC, compiles in Java, and creates source code files and executables in temporary folder.\n"

    main_head = "#include <stdio.h>\n" \
            "#include <math.h>\n" \
            "int main(int argc, char* argv[], char** env){\n"

    main_foot = "\nreturn 0;\n}"

##//%include:src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(JavaKernel, self).__init__(*args, **kwargs)
        self.kernelinfo='[MyC Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        
#################
##%include:src/compile_with_javac.py
##%include:src/j_exec_javac_.py
##%include:src/do_jv_runcode.py
##%include:src/do_jv_compilecode.py
##%include:src/do_jv_create_codefile.py
##%include:src/do_jv_preexecute.py



[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Java/../../../jupyter-MyJava-kernel/jupyter_MyJava_kernel/kernel.py created successfully
